<h1 align="center">Long Short-Term Memory (LSTM) Networks </h1> 

### The Problem of Long-Term Dependencies

One of the appeals of RNNs is the idea that they might be able to connect previous information to the present task, such as using previous video frames might inform the understanding of the present frame. If RNNs could do this, they’d be extremely useful. But can they? It depends.

Sometimes, we only need to look at recent information to perform the present task. For example, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in “the clouds are in the sky,” we don’t need any further context – it’s pretty obvious the next word is going to be sky. In such cases, where the gap between the relevant information and the place that it’s needed is small, RNNs can learn to use the past information.

But there are also cases where we need more context. Consider trying to predict the last word in the text “I grew up in France… I speak fluent French.” Recent information suggests that the next word is probably the name of a language, but if we want to narrow down which language, we need the context of France, from further back. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large. Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

![title](../images/long_term_dependencies.jpg)

### LSTM Networks

LSTM are a special kind of RNN, capable of learning long-term dependencies. LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn! All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer (figure below, left):

In [1]:
function mlp1_rnn(param, hₜ₋, xₜ)
    input  = hcat(hₜ₋, xₜ)
    hₜ = tanh(input * param[1] .+ param[2])
    return (hₜ, xₜ)
end

mlp1_rnn (generic function with 1 method)

LSTMs also have this chain like structure, but the repeating module has a different structure. Instead of having a single neural network layer, there are four, interacting in a very special way (figure below, right):

![title](../images/RNN_vs_LSTM.jpg)

In the above diagram, each line carries an entire vector, from the output of one node to the inputs of others. The pink circles represent pointwise operations, like vector addition, while the yellow boxes are learned neural network layers. Lines merging denote concatenation, while a line forking denote its content being copied and the copies going to different locations.

### The Core Idea Behind LSTMs

The key to LSTMs is the cell state $C$, the horizontal line running through the top of the diagram: entering from the top left as $C_{t-1}$ and exiting from the top right as $C_t$.

The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions (vector addition and multiplication). It’s very easy for information to just flow along it unchanged. The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates. Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation (first, second, and fourth learned layers). The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!”

An LSTM has three of these gates, to protect and control the cell state.

### Step-by-Step LSTM Walk Through

The first step in our LSTM is to decide what information we’re going to throw away from the cell state. This decision is made by a sigmoid layer called the “forget gate layer” and it's denoted as $f$. It looks at $h_{t-1}$ and $x_t$, and outputs a number between 0 and 1 for each number in the cell state $C_{t−1}$. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.” Formally:

\begin{equation}
f_t = \sigma\big(W_f\, [h_{t-1}\,\,x_t] + b_f\big)
\end{equation}


Let’s go back to our example of a language model trying to predict the next word based on all the previous ones. In such a problem, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.

The next step is to decide what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the “input gate layer” (denoted by $i$) decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values (denoted by $C$), $\tilde{C}_t$, that could be added to the state. In the next step, we’ll combine these two to create an update to the state. Formally:

\begin{equation}
i_t = \sigma\big(W_i\, [h_{t-1}\,\,x_t] + b_i\big)\hspace{.7cm}
\end{equation}
\begin{equation}
\tilde{C}_t = \tanh\big(W_C\, [h_{t-1}\,\,x_t] + b_C\big)
\end{equation}


In the example of our language model, we’d want to add the gender of the new subject to the cell state, to replace the old one we’re forgetting.

It’s now time to update the old cell state, $C_{t−1}$, into the new cell state $C_t$. The previous steps already decided what to do, we just need to actually do it. We multiply the old state by $f_t$, forgetting the things we decided to forget earlier. Then we add $i_t∗\tilde{C}_t$. This is the new candidate values, scaled by how much we decided to update each state value. Formally:

\begin{equation}
C_t = f_t*C_{t-1} + i_t*\tilde{C}
\end{equation}


In the case of the language model, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output and denote the output by $o$. Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to. Formally:

\begin{equation}
o_t = \sigma\big(W_o\, [h_{t-1}\,\,x_t] + b_o]\big)
\end{equation}
\begin{equation}
h_t = o_t * \tanh C_t\hspace{1.8cm}
\end{equation}


For the language model example, since it just saw a subject, it might want to output information relevant to a verb, in case that’s what is coming next. For example, it might output whether the subject is singular or plural, so that we know what form a verb should be conjugated into if that’s what follows next.

### Embeddings

In mathematics, an [embedding](https://en.wikipedia.org/wiki/Embedding) is one instance of some mathematical structure contained within another instance, such as a group that is a subgroup. When some object $X$ is said to be embedded in another object $Y$, the embedding is given by some *injective* and *structure-preserving* map $f : X \rightarrow Y$. The precise meaning of "structure-preserving" depends on the kind of mathematical structure of which $X$ and $Y$ are instances. In the terminology of category theory, a structure-preserving map is called a morphism. The fact that a map $f : X \rightarrow Y$ is an embedding is often indicated by the use of a "hooked arrow": $f:X\hookrightarrow Y$. 

Word embeddings allow us to perform computations on an otherwise abstract input (e.g. how can we compare 'a' and '\n')? For example, suppose that we are given the task of determining the likelihood that a given document was plagiarized. That means that we need to compare this document to multiple to others and determine how *related* or equal the documents are. One scheme could involve comparing the words found on each document. We can represent a word as a vector of numbers:

\begin{equation}
\text{banana} = (0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 1, 0)
\end{equation}

The actual interpretation of the vector varies. The vector could represent the documents in which the words occurs, in this case ocurring at every document $i$ such that $\text{banana}(i) = 1$. Another representation that takes into account local context is neighboring word context, e.g. $banana$ is found in the sentense

\begin{equation}
"\underset{-1}{\text{yellow}}\,\underset{+0}{\text{banana}}\,\underset{+1}{\text{grows}}\,\underset{+2}{\text{on}}
\,\underset{+2}{\text{trees}}\,\underset{+2}{\text{in}}\,\underset{+2}{\text{africa}}"
\end{equation}

Comparing two vectors (e.g. using cosine similarity) estimates how similar the two words are. However, *the notion of relatedness* depends on what vector representation you have chosen for words. For example, is Seattle similar to Denver because they are both cities? What about Seattle and Seahawks? In addition, these vectors often tend to be very high-dimensional (thousands, or even millions) and sparse. Thus, we must turn to "learning" a particular lower-dimensional dense embedding (or vector) for our task. You can learn more about embeddings [here](https://stats.stackexchange.com/questions/182775/what-is-an-embedding-layer-in-a-neural-network) or [here](https://www.slideshare.net/BhaskarMitra3/a-simple-introduction-to-word-embeddings).

$\textbf{Embeddings in LSTM}$ Suppose that the inputs to our LSTM network are characters from a particular vocabulary $vocab$. The idea is that since our vocabulary is discrete, we will learn a map, $emb$, which will embed each character into a continuous vector space $X$. Using this vector space representation will allow us to have a continuous, distributed representation of our vocabulary characters. If for example the lenght of our vocabulary is $V$, we may now use our continuous character features to create a distributed representation of our $V$ characters.  In the process of training a language model we will learn this character embedding map $emb:voca\hookrightarrow X$. The hope is that by using a continuous representation, our embedding will map similar words to similar regions.

Formally, we will learn a mapping (embedding) $emb:vocab\hookrightarrow X$ that maps the space $vocab$, corresponding to a vocabulary with $V$ characters, to a continuous vector space $X$. We denote a vector from this vector space as $x\in X$. 

## Julia Implementation

In [17]:
using Knet, AutoGrad
using Knet: sigm_dot, tanh_dot

#### Load input data and define model parameters

Our first goal is to define the models parameters (e.g. layer length and size, embedding vector size) and to load/pre-process the input data. First we define the model parameters:

In [27]:
seed       = -1 
H          = [128];
embed_size = 168;
datafiles  = ["input.txt"];

In [28]:
seed > 0 && srand(seed);
text = map(readstring, datafiles);

In summary, we have the following equations for a single-layer LSTM network with input $x_t$:

\begin{equation}
\begin{aligned}
f_t &= \sigma\big(W_f\, [h_{t-1}\,\,x_t] + b_f\big) & (1) &\, \text{Forget gate} \\
i_t &= \sigma\big(W_i\, [h_{t-1}\,\,x_t] + b_i\big) & (2) &\, \text{Input gate} \\
\tilde{C}_t &= \tanh\big(W_C\, [h_{t-1}\,\,x_t] + b_C\big) & (3) &\, \text{Cell candidate} \\
C_t &= f_t*C_{t-1} + i_t*\tilde{C} & (4) &\, \text{New cell}\\
o_t &= \sigma\big(W_o\, [h_{t-1}\,\,x_t] + b_o]\big) & (5) &\, \text{Output gate} \\
h_t &= o_t * \tanh C_t & (6) &\, \text{New output} 
\end{aligned}
\end{equation}

Let us consider the size of all the variables involved. The size of the input vector $x_t$ is denoted as $1\times I$, while the size of $h_{t-1}$ and $C_t$ are both denoted as $1\times H$ (the size of the lstm layer), therefore the concatenated input $[h_{t-1}\,\,x_t]$ has a size $1\times (H+I)$. In practice, the weight matrices $W_f, W_i, W_C$ and $W_o$ are combined into a single weight matrix $W$, therefore the size of the combined matrix $W$ is $(H+I)\times 4H$ and the size of $b$ is $1\times 4H$. 

For multiple layers, the input to an lstm layer $l$ is the hidden state from the previous layer, i.e. $h^{l-1}$, therefore $[h_{t-1}^l\,\,h_t^{l-1}]$ has size $1\times (H^{l-1} + H^l)$, $W^l$ has size $(H^{l-1} + H^l)\times 4H^l$, and $b^l$ has size $1\times 4H^l$. 

Layer $l=0$ is simply the input (embedding) layer:

\begin{equation}
h^0_t = emb * x_t
\end{equation}

where $emb$ is the embedding matrix of size $V\times H^0$, V is the size of the vocabulary, and $H^0$ is the size of the embedding vector. The input vector $x_t$ is a one-hot vector of size $V$. 

The ouput (readout, or prediction layer) is given by:

\begin{equation}
y = ph + b_p
\end{equation}

where $p$, $b_p$ denote the weights (matrix) and biases of the prediction layer. If the $L$ is the number of lstm layers, the size of the output vector $y$ must be $V$ (the size of the vocabulary) such that $p$ has size $H^L\times V$.

We summarize what we have done so far graphically:

<img src="../images/lstm_vocab.jpg" width="800">

#### LSTM Layer:

The function lstm below takes as input the weights/biases, the previous hidden/cell states, and the actual current input:

In [21]:
function lstm(weight, bias, hₜ₋, Cₜ₋, xₜ)
    
    H         = size(hₜ₋, 2)
    pre_gates = hcat(xₜ, hₜ₋) * weight .+ bias
    fₜ        = sigm_dot(pre_gates[:, 1:H])       # (1) forget gate
    iₜ        = sigm_dot(pre_gates[:, 1 + H:2H])  # (2) input gate
    oₜ        = sigm_dot(pre_gates[:, 1 + 2H:3H]) # (5) output gate
    changeₜ   = tanh_dot(pre_gates[:, 1 + 3H:4H]) # (3) cell candidate

    Cₜ = fₜ .* Cₜ₋ + iₜ .* changeₜ                # (4) new cell 
    hₜ = oₜ .* tanh_dot(Cₜ)                      # (6) new output

    return (hₜ, Cₜ)

end

lstm (generic function with 1 method)

#### Weight Initialization

Suppose that we have an LSTM network with $L$ layers. We need to be able to initialize all weights (params) in the model. Here we initialize all weights with $xavier$ initialization into the variable "model" of size $2L + 3$: The last three elements of model are the embedding matrix and the weight/bias for the final prediction.

In [2]:
"""
    initweights(H, V, embed_size)

Initialize the weights of an lstm network for character prediction such that L = length(H) is
the number of layers, H[k] is the size of the hidden layer k, V is the size of the vocabulary,
and embed_size is the size of the embedding vector. All weights are initialize with xavier 
initialization and all biases are initialize to zero. 

model = initweights(...) is a vector with wₖ = model[2k - 1], bₖ = [2k] corresponding to the 
weight and biases of layer k, emb = model[end - 2] corresponding to the embedding matrix, and 
p = model[end - 1], bₚ = model[end] corresponding to the weight and bias of the final prediction layer. 

With Hₖ = H[k] as the size of layer k, wₖ and bₖ are of size (Hₖ₋ + Hₖ, 4Hₖ) and (1, 4Hₖ) accordingly,
such with H₀ = embed_size. 

"""

function initweights(H, V, embed_size)
    init(d...) = xavier(d...)
    bias(d...) = zeros(d...)
    model      = Vector{Any}(2 * length(H) + 3)
    
    Hₖ₋ = embed_size
    for k = 1:length(H)
        # Your code starts here
        Hₖ = H[k]
        model[2k - 1] = init(Hₖ₋ + Hₖ, 4Hₖ)
        model[2k]     = bias(1, 4Hₖ)
        Hₖ₋ = Hₖ
        # Your code ends here
    end
    
    model[end - 2] = init(V, embed_size)
    model[end - 1] = init(H[end], V)
    model[end]     = bias(1, V)
    
    return model
end

initweights

#### Create Initial state

 At each time step, we take the hidden state from previous time step as input. To be able to do that, first we need to initialize hidden state. We also store updated hidden states in array created here. We initialize state as a zero matrix.

In [3]:
let blank = nothing; global initstate
    
    """
    initstate(model, batch_size)

    Initialize the state on an lstm network such that h[k], C[k] = state[k] are the hidden
    and cell state of layer k. 

    """
    function initstate(model, batch_size)

        L     = div(length(model) - 3, 2)
        state = Vector{Any}(2 * L)
        for k = 1:L
            # Your code starts here    
            bₖ = model[2k]
            Hₖ = div(length(bₖ), 4)
            if typeof(blank)!=typeof(bₖ) || size(blank)!=(Hₖ, batch_size)
                blank = fill!(similar(bₖ, batch_size, Hₖ), 0)
            end
            state[2k-1] = state[2k] = blank # hₖ = state[2k-1], cₖ = state[2k]
            # Your code ends here
            end
        return state
    end
end

initstate (generic function with 1 method)

#### Create Predict function

Notice below that we are not performing the initial embedding operation xₜ = xₜ * model[end - 2]. Instead, we will see that it's more efficient to simply select the respective elements from the embedding matrix. In addition, note that we don't actually run the final prediction layer yₜ = xₜ * model[end-1] .+ model[end] either.

In [5]:
function predict(model, stateₜ₋, xₜ; pdrop=0)
    L = div(length(model) - 3, 2)
    stateₜ = similar(stateₜ₋)
    
    for k = 1:L
        # Your code starts here
        (stateₜ[2k - 1], stateₜ[2k]) = lstm(model[2k - 1], model[2k], state[2k - 1], state[2k], xₜ)
        xₜ = stateₜ[2k - 1]
        # Your code ends here
    end

    return xₜ, stateₜ
end

predict (generic function with 1 method)

#### Generate and Sample function

Generate function is a function we use to create some text that is similar to our training data. We provide sample function to you. You can predict the next character by using sample function once you calculate the probabilities given the input. index to char is the same dictionary as you created with createdictionary function but it works in the reverse direction. It gives you the character given the index.

In [11]:
function createVocabulary(text)
    vocab = Dict{Char,Int}()
    # Your code starts here
    idx = 1
    for char in text[1]
        try vocab[char] catch vocab[char] = idx; idx += 1; end
    end
    # Your code ends here
    return vocab
end

createVocabulary (generic function with 1 method)

In [12]:
function generate(model, tok2int, nchar)
    int2tok = Vector{Char}(length(tok2int))
    for (k,v) in tok2int; int2tok[v] = k; end
    input = tok2int[' ']
    state = initstate(model, 1)
    for t in 1:nchar
        embed = model[end-2][[input],:]
        ypred,state = predict(model,state,embed)
        ypred = ypred * model[end-1] .+ model[end]
        input = sample(exp.(logp(ypred)))
        print(int2tok[input])
    end
    println()
end

function sample(p)
    p = convert(Array,p)
    r = rand()
    for c = 1:length(p)
        r -= p[c]
        r < 0 && return c
    end
end

sample (generic function with 1 method)

In [13]:
seed       = -1 
H          = [128];
embed_size = 168;
datafiles  = ["input.txt"]

1-element Array{String,1}:
 "input.txt"

In [15]:
vocab = createVocabulary(text);

In [18]:
model = initweights(H, length(vocab), embed_size);

In [19]:
state = initstate(model,1);

In [22]:
println("########## RANDOM MODEL OUTPUT ############")
generate(model, vocab, 500) ## change togenerate if you want longer sample text

########## RANDOM MODEL OUTPUT ############
G'b\.cGaoV2v9jk1V,Q97IVOw6NuFifR;lz& jyH.-kepcqEWOghb{Pl'fHheSDzpQ{H7Tl ja9l1.TBOx:z:htoGdC,GoH&&MthGh,21wH97*JD6n9*eoN:JSd{L{5bKI?nI:*mYK3fMAbR!,C3H&L723SQOzC}6uvpU2w0leTn\\cf1&UjUJ.cMCobRsNtiBvBb}Yt\sInlxJ9PLjy4-O!h2Q:1HRNAvj\}FsInLd1FrIHFEAe7\v8h\:ojytpkaTDi8Wa.mM3U qIyOJ-,7!9RmwT3\nrr}WG2TeWS*9C
QIdK!T?Duw-BvoH!f}toei{JmGb*Jw-d;D}yAqcEKB2D3\nsGj5OPi T-i2z9MJBNMbT6xHlw3*Qe0HQwhxgx,6;kYp1ex}oU31l.02fptQ1*7Nd6s;Wu2-{gl??Ee:'-n-c-
}gYG2im2aMY.*oTqnU{Iywe M'bWsVtDwU-3jQhe7c'Q3SO&KMI.LwvQRpI{UwzIk


### Loss function

In [15]:
function loss(model, state, sequence, range=1:length(sequence)-1; newstate=nothing, pdrop=0.0)
    preds = []
    for t in range
        input = model[end-2][sequence[t],:]
        pred,state = predict(model,state,input; pdrop=pdrop)
        push!(preds, pred)
    end
    
    if newstate != nothing
        copy!(newstate, map(AutoGrad.getval,state))
    end
    
    pred0 = vcat(preds...)
    pred1 = dropout(pred0,pdrop)
    pred2 = pred1 * model[end-1]
    pred3 = pred2 .+ model[end]
    logp1 = logp(pred3,2)
    nrows,ncols = size(pred3)
    golds = vcat(sequence[range[1]+1:range[end]+1]...)
    index = similar(golds)
    @inbounds for i=1:length(golds)
        index[i] = i + (golds[i]-1)*nrows
    end
    logp2 = logp1[index]
    logp3 = sum(logp2)
    return -logp3 / length(golds)
end

# Knet magic
lossgradient = grad(loss)

function avgloss(model, sequence, S)
    
    T = length(sequence)
    B = length(sequence[1])
    state = initstate(model, B)
    total = count = 0
    for i in 1:S:T-1
        j = min(i+S-1,T-1)
        n = j-i+1
        total += n * loss(model, state, sequence, i:j; newstate=state)
        count += n
    end
    return total / count
end

avgloss (generic function with 1 method)

In [30]:
function train(model, sequence, optim, S; pdrop=0.0)
    T = length(sequence)
    B = length(sequence[1])
    
    state = initstate(model, B)
    for i in 1:S:T-1
        # Your code starts here
        j = min(i + S - 1, T - 1)
        grads = lossgradient(model, state, sequence, i:j, pdrop=pdrop)
        update!(model, grads, optim)
        # Your code ends here
    end
end

train (generic function with 1 method)

In [31]:
function minibatch(chars, tok2int, batch_size)
    chars  = collect(chars)
    nbatch = div(length(chars), batch_size)
    data   = [zeros(Int, batch_size) for i=1:nbatch]
    for n = 1:nbatch
        for b = 1:batch_size
            char = chars[(b-1)*nbatch + n]
            data[n][b] = tok2int[char]
        end
    end
    return data
end

minibatch (generic function with 1 method)

105989

In [33]:
data = map(t->minibatch(t, vocab, 1), text);

In [85]:
d = data_one_hots(data[1], length(vocab));

In [88]:
data[1][1]

1-element Array{Int64,1}:
 1